In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI：追踪本地训练模型的参数和指标

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_local_trained_models.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexperiments%2Fcomparing_local_trained_models.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab企业版中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/comparing_local_trained_models.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/comparing_local_trained_models.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

作为数据科学家，您可能会在笔记本上开始本地运行模型实验。根据您使用的框架，您需要跟踪参数，训练时间序列和评估指标。通过这种方式，您可以解释您选择的建模方法。

了解有关[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments)的更多信息。

### 目标

在本教程中，您将学习如何使用Vertex AI Experiments来比较和评估模型实验。

本教程使用以下谷歌云ML服务和资源：

- Vertex AI Workbench
- Vertex AI Experiments

执行的步骤包括：

- 记录模型参数
- 在每个epoch上将损失和指标记录到TensorBoard
- 记录评估指标

### 数据集

在这个笔记本中，您将训练一个简单的分布式神经网络（DNN）模型，根据[auto-mpg数据集](https://www.kaggle.com/devanshbesain/exploration-and-analysis-auto-mpg)中的汽车信息来预测汽车每加仑的里程。

成本

本教程使用了谷歌云的计费组件：

* Vertex AI
* Vertex AI TensorBoard
* 云存储

了解[Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing)和[云存储的定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的包

In [ ]:
! pip3 install --upgrade tensorflow==2.8 \
                        protobuf==3.20.3 \
                        google-cloud-aiplatform \
                        matplotlib \
                        pandas \
                        'numpy<2' -q --no-warn-conflicts

### 重新启动运行时 (仅限Colab)

要使用新安装的包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。请等待直到完成后再继续下一步。⚠️</b>
</div>

### 在您的笔记本环境上进行身份验证（仅限Colab）

在Google Colab上对您的环境进行身份验证。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

将Google Cloud项目信息设置并初始化Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目并 [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于 [设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import uuid

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.cloud import aiplatform as vertex_ai
from tensorflow.python.keras import Sequential, layers
from tensorflow.python.keras.utils import data_utils

### 定义常量

In [ ]:
EXPERIMENT_NAME = "[your-experiment-name]"  # @param {type:"string"}

如果EXPERIMENT_NAME未设置，则在下方设置一个默认名称:

In [ ]:
if EXPERIMENT_NAME == "[your-experiment-name]" or EXPERIMENT_NAME is None:
    EXPERIMENT_NAME = f"my-experiment-{uuid.uuid1()}"

### 初始化Python版的Vertex AI SDK

为您的项目和对应的存储桶初始化Python版的Vertex AI SDK。

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 使用支持Vertex AI TensorBoard创建Vertex AI实验

您可以使用init()方法创建Vertex AI实验。这将自动获取或创建默认的Vertex AI TensorBoard，并将其与您的实验关联起来。

了解更多有关[TensorBoard概述](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview)。

In [ ]:
vertex_ai.init(experiment=EXPERIMENT_NAME)

## 使用 Vertex AI 实验进行训练

Vertex AI允许用户跟踪实验运行的步骤（例如，预处理、训练），并跟踪这些步骤的输入（例如，算法、参数、数据集）和输出（例如，模型、检查点、指标）。

为了更好地理解参数和指标是如何存储和组织的，以下概念进行了解释：

1. **实验**描述了一个上下文，将您的运行和创建的工件组合成一个逻辑会话。例如，在这个笔记本中，您创建了一个实验，并将数据记录到该实验中。

2. **运行**表示您在执行实验时执行的单个路径/方法。一个运行包括您用作输入或输出的工件，以及您在此执行中使用的参数。一个实验可以包含多个运行。

您可以使用Python的Vertex AI SDK跟踪每个实验中本地训练的模型的指标和参数，跨多个实验运行。

在以下示例中，您将训练一个简单的分布式神经网络（DNN）模型，以根据[auto-mpg数据集](https://www.kaggle.com/devanshbesain/exploration-and-analysis-auto-mpg)中的汽车信息预测汽车每加仑的英里数（MPG）。

In [ ]:
# Helpers ----------------------------------------------------------------------


def read_data(uri):
    """
    Read data
    Args:
        uri: path to data
    Returns:
        pandas dataframe
    """
    dataset_path = data_utils.get_file("auto-mpg.data", uri)
    column_names = [
        "MPG",
        "Cylinders",
        "Displacement",
        "Horsepower",
        "Weight",
        "Acceleration",
        "Model Year",
        "Origin",
    ]
    raw_dataset = pd.read_csv(
        dataset_path,
        names=column_names,
        na_values="?",
        comment="\t",
        sep=" ",
        skipinitialspace=True,
    )
    dataset = raw_dataset.dropna()
    dataset["Origin"] = dataset["Origin"].map(
        lambda x: {1: "USA", 2: "Europe", 3: "Japan"}.get(x)
    )
    dataset = pd.get_dummies(dataset, prefix="", prefix_sep="", dtype=float)
    return dataset


def train_test_split(dataset, split_frac=0.8, random_state=0):
    """
    Split data into train and test
    Args:
        dataset: pandas dataframe
        split_frac: fraction of data to use for training
        random_state: random seed
    Returns:
        train and test dataframes
    """
    train_dataset = dataset.sample(frac=split_frac, random_state=random_state)
    test_dataset = dataset.drop(train_dataset.index)
    train_labels = train_dataset.pop("MPG")
    test_labels = test_dataset.pop("MPG")

    return train_dataset, test_dataset, train_labels, test_labels


def normalize_dataset(train_dataset, test_dataset):
    """
    Normalize data
    Args:
        train_dataset: pandas dataframe
        test_dataset: pandas dataframe

    Returns:

    """
    train_stats = train_dataset.describe()
    train_stats = train_stats.transpose()

    def norm(x):
        return (x - train_stats["mean"]) / train_stats["std"]

    normed_train_data = norm(train_dataset)
    normed_test_data = norm(test_dataset)

    return normed_train_data, normed_test_data


def build_model(num_units, dropout_rate):
    """
    Build model
    Args:
        num_units: number of units in hidden layer
        dropout_rate: dropout rate
    Returns:
        compiled model
    """
    model = Sequential(
        [
            layers.Dense(
                num_units,
                activation="relu",
                input_shape=[9],
            ),
            layers.Dropout(rate=dropout_rate),
            layers.Dense(num_units, activation="relu"),
            layers.Dense(1),
        ]
    )

    model.compile(loss="mse", optimizer="adam", metrics=["mae", "mse"])
    return model


def train(
    model,
    train_data,
    train_labels,
    validation_split=0.2,
    epochs=10,
):
    """
    Train model
    Args:
        train_data: pandas dataframe
        train_labels: pandas dataframe
        model: compiled model
        validation_split: fraction of data to use for validation
        epochs: number of epochs to train for
    Returns:
        history
    """
    history = model.fit(
        train_data, train_labels, epochs=epochs, validation_split=validation_split
    )

    return history

运行实验并评估实验运行

您定义多个实验配置，运行实验并在Vertex AI实验中跟踪它们。

In [ ]:
# Define experiment parameters
parameters = [
    {"num_units": 16, "dropout_rate": 0.1, "epochs": 3},
    {"num_units": 16, "dropout_rate": 0.1, "epochs": 10},
    {"num_units": 16, "dropout_rate": 0.2, "epochs": 10},
    {"num_units": 32, "dropout_rate": 0.1, "epochs": 10},
    {"num_units": 32, "dropout_rate": 0.2, "epochs": 10},
]

# Read data
dataset = read_data(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
)

# Split data
train_dataset, test_dataset, train_labels, test_labels = train_test_split(dataset)

# Normalize data
normed_train_data, normed_test_data = normalize_dataset(train_dataset, test_dataset)

# Run experiments
for i, params in enumerate(parameters):

    # Initialize Vertex AI Experiment run
    vertex_ai.start_run(run=f"auto-mpg-local-run-{i}")

    # Log training parameters
    vertex_ai.log_params(params)

    # Build model
    model = build_model(
        num_units=params["num_units"], dropout_rate=params["dropout_rate"]
    )

    # Train model
    history = train(
        model,
        normed_train_data,
        train_labels,
        epochs=params["epochs"],
    )

    # Log additional parameters
    vertex_ai.log_params(history.params)

    # Log metrics per epochs
    for idx in range(0, history.params["epochs"]):
        vertex_ai.log_time_series_metrics(
            {
                "train_mae": history.history["mae"][idx],
                "train_mse": history.history["mse"][idx],
            }
        )

    # Log final metrics
    loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
    if np.isnan(loss):
        loss = 0
    if np.isnan(mae):
        mae = 0
    if np.isnan(mse):
        mse = 0
    vertex_ai.log_metrics({"eval_loss": loss, "eval_mae": mae, "eval_mse": mse})

    vertex_ai.end_run()

将参数和指标提取到数据框中进行分析

我们还可以将与任何实验相关的所有参数和指标提取到数据框中进行进一步分析。

In [ ]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df.T

### 可视化实验的参数和指标

您使用平行坐标绘图来可视化实验的参数和指标

In [ ]:
plt.rcParams["figure.figsize"] = [15, 5]

ax = pd.plotting.parallel_coordinates(
    experiment_df.reset_index(level=0),
    "run_name",
    cols=[
        "param.num_units",
        "param.dropout_rate",
        "param.epochs",
        "metric.eval_loss",
        "metric.eval_mse",
        "metric.eval_mae",
    ],
    color=["blue", "green", "pink", "red"],
)
ax.set_yscale("symlog")
ax.legend(bbox_to_anchor=(1.0, 0.5))

在Cloud控制台中可视化实验

运行以下命令以获取您项目的Vertex AI实验URL。

In [ ]:
print("Vertex AI Experiments:")
print(
    f"https://console.cloud.google.com/ai/platform/experiments/experiments?folder=&organizationId=&project={PROJECT_ID}"
)

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

或者，您可以删除本教程中创建的各个资源。

In [ ]:
# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
backing_tensorboard = exp.get_backing_tensorboard_resource()
exp.delete(delete_backing_tensorboard_runs=True)

# Delete Tensorboard
delete_tensorboard = False  # Set True for deletion

if delete_tensorboard:
    backing_tensorboard.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False  # Set True for deletion

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}